In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextEmbedding,
)

#from psycopg import Cursor
#from psycopg.sql import SQL, Identifier
#from psycopg_pool import ConnectionPool
from semantic_kernel.connectors.memory.postgres.postgres_memory_store import (
    PostgresMemoryStore,
)

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
endpoint = os.getenv("GPT_OPENAI_ENDPOINT")
api_key = os.getenv("GPT_OPENAI_KEY")
gpt_deployment_name = os.getenv("GPT_OPENAI_DEPLOYMENT_NAME")
conn_str = os.getenv("PG_CONN_STR_PY")
ada_deployment_name = "ada"

In [3]:
kernel = sk.Kernel()
azure_chat_service = AzureChatCompletion(deployment_name=gpt_deployment_name, endpoint=endpoint, api_key=api_key)
azure_text_embedding = AzureTextEmbedding(deployment_name=ada_deployment_name, endpoint=endpoint, api_key=api_key)
kernel.add_chat_service("chat_completion", azure_chat_service)
kernel.add_text_embedding_generation_service("ada", azure_text_embedding)

mem_store = PostgresMemoryStore(conn_str,1536,1,3)
kernel.register_memory_store(memory_store=mem_store)
kernel.import_plugin(sk.core_plugins.TextMemoryPlugin(), "text_memory")

KernelPlugin(name='text_memory', description=None, functions={'recall': KernelFunction(plugin_name='text_memory', description='Recall a fact from the long term memory', name='recall', is_semantic=False, stream_function=<bound method TextMemoryPlugin.recall of TextMemoryPlugin()>, parameters=[ParameterView(name='input', description='The information to retrieve', default_value='', type_='string', required=False), ParameterView(name='limit', description='The maximum number of relevant memories to recall.', default_value='1', type_='string', required=False), ParameterView(name='relevance', description='The relevance score, from 0.0 to 1.0; 1.0 means perfect match', default_value='0.75', type_='string', required=False), ParameterView(name='collection', description='The collection to search for information', default_value='generic', type_='string', required=False)], delegate_type=<DelegateTypes.InStringAndContextOutTaskString: 13>, function=<bound method TextMemoryPlugin.recall of TextMemory

In [4]:
def read_file(file: str)->str:
    with open(file, "r") as f:
        return f.read()
    
def ingest_content(path:str):
    import os
    chunks = []
    files = os.listdir(path)
    for f in files:
        if f.endswith("water.txt"):            
            content = read_file("data/"+f)
            paragraphs = content.split("\n\n")
            l = len(paragraphs)
            id = 1
            for p in paragraphs:
                lid = f"{f}-{l}-{id}"
                c = {"id":lid,"chunk":p,"file":f}
                chunks.append(c)
                id += 1
    return chunks
    #return {f: read_file(path + "/" + f) for f in files}

chunks = ingest_content("data")

In [5]:
COLLECTION_NAME = "PYCollection"
async def populate_memory(kernel: sk.Kernel, chunks: list) -> None:
    for chunk in chunks:
        await kernel.memory.save_information(collection=COLLECTION_NAME, id=chunk["id"], text=chunk["chunk"], description=chunk["file"])

In [6]:
await populate_memory(kernel, chunks)

In [11]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "What is the chemical composition of water?",
    ]

    for question in questions:
        print(f"Question: {question}\n")
        result = await kernel.memory.search(COLLECTION_NAME, question,1,0.8)
        print(f"Answer: {result[0].text}\n")
        print(f"File: {result[0].description}\n")

In [10]:
await search_memory_examples(kernel)

Question: What is the chemical composition of water?
Answer: Water is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless,[c] and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms (in which it acts as a solvent[19]). It is vital for all known forms of life, despite not providing food energy or organic micronutrients. Its chemical formula, H2O, indicates that each of its molecules contains one oxygen and two hydrogen atoms, connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°.[20] "Water" is also the name of the liquid state of H2O at standard temperature and pressure.

File: wikipedia-water.txt

